In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-08-27T17:00:53.245832-04:00

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.28.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.11.0-37-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

In [4]:
%watermark --iversions

xgboost: 1.6.2
logging: 0.5.1.2
pandas : 1.4.3
optuna : 2.10.1
dask   : 2022.7.1
numpy  : 1.22.4



In [5]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [6]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2022-08-27 17:01:05,935 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-08-27 17:01:05,935 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-08-27 17:01:05,939 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-08-27 17:01:05,939 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [7]:
!nvidia-smi

Sat Aug 27 17:01:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   35C    P2    64W / 280W |    167MiB / 24220MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [8]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [9]:
train = pd.read_csv('../input/train.csv.gz')

target = train['label'].values
target

array([2, 9, 6, ..., 8, 8, 7])

In [10]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective':'multi:softmax', 
        'num_class':10,
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'random_state': 137,

    }
    
    #start_time = time()
    kf = StratifiedKFold(5, shuffle=True, random_state=1974)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i], train_ys[i])
        dval = xgb.dask.DaskDMatrix(client, val_folds[i], val_ys[i])
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()


    acc = accuracy_score(target, train_oof)
    
    return acc

In [11]:
train_oof.shape

(60000,)

In [12]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_0.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_mnist_0.db", study_name="five_fold_optuna_xgb_0", direction='maximize')

In [13]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[17:01:30] task [xgboost.dask-0]:tcp://127.0.0.1:40161 got new rank 0
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[17:02:22] task [xgboost.dask-1]:tcp://127.0.0.1:41589 got new rank 0
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[17:03:12] task [xgboost.dask-0]:tcp://127.0.0.1:40161 got new rank 0
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[17:04:02] task [xgboost.dask-1]:tcp://127.0.0.1:41589 got new rank 0
/home/tunguz

CPU times: user 30 s, sys: 13 s, total: 43.1 s
Wall time: 13min 20s


In [14]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.866717,3.566109,0.336990,logloss,0.110663,0.080941,17,289,0.400894,gpu_hist,COMPLETE
1,1,0.885717,0.003675,0.470387,logloss,0.004926,0.087824,13,294,0.759316,gpu_hist,COMPLETE
2,2,0.887100,0.732165,0.957175,logloss,0.001125,0.045880,9,286,0.947977,gpu_hist,COMPLETE


In [15]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0.csv', index=False)
df.head()

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[17:14:51] task [xgboost.dask-1]:tcp://127.0.0.1:41589 got new rank 0
2022-08-27 17:17:48,851 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-27 17:17:49,017 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[17:17:51] task [xgboost.dask-0]:tcp://127.0.0.1:40161 got new rank 0
2022-08-27 17:20:45,774 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-27 17:20:45,929 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (th

CPU times: user 1min 53s, sys: 1min 1s, total: 2min 54s
Wall time: 1h 9min 39s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.866717,3.566109,0.336990,logloss,0.110663,0.080941,17,289,0.400894,gpu_hist,COMPLETE
1,1,0.885717,0.003675,0.470387,logloss,0.004926,0.087824,13,294,0.759316,gpu_hist,COMPLETE
2,2,0.887100,0.732165,0.957175,logloss,0.001125,0.045880,9,286,0.947977,gpu_hist,COMPLETE
3,3,0.880850,0.097329,0.357466,logloss,1.206300,0.004286,17,38,0.671190,gpu_hist,COMPLETE
4,4,0.866200,0.008397,0.573746,logloss,0.202615,0.006200,9,205,0.932661,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=10)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0.csv', index=False)
df.head()

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:24:31] task [xgboost.dask-0]:tcp://127.0.0.1:40161 got new rank 0
2022-08-27 18:25:45,307 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-27 18:25:45,465 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:25:48] task [xgboost.dask-1]:tcp://127.0.0.1:41589 got new rank 0
2022-08-27 18:27:00,688 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-27 18:27:00,845 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (th

CPU times: user 4min 5s, sys: 2min 11s, total: 6min 16s
Wall time: 2h 29min 52s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.866717,3.566109,0.336990,logloss,0.110663,0.080941,17,289,0.400894,gpu_hist,COMPLETE
1,1,0.885717,0.003675,0.470387,logloss,0.004926,0.087824,13,294,0.759316,gpu_hist,COMPLETE
2,2,0.887100,0.732165,0.957175,logloss,0.001125,0.045880,9,286,0.947977,gpu_hist,COMPLETE
3,3,0.880850,0.097329,0.357466,logloss,1.206300,0.004286,17,38,0.671190,gpu_hist,COMPLETE
4,4,0.866200,0.008397,0.573746,logloss,0.202615,0.006200,9,205,0.932661,gpu_hist,COMPLETE


In [17]:
%%time
study.optimize(objective, n_trials=40)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0.csv', index=False)
df.head()

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[22:39:28] task [xgboost.dask-0]:tcp://127.0.0.1:40161 got new rank 0
2022-08-27 22:40:42,539 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-27 22:40:42,712 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[22:40:45] task [xgboost.dask-1]:tcp://127.0.0.1:41589 got new rank 0
2022-08-27 22:41:58,606 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (threshold: 10%)
2022-08-27 22:41:58,778 - distributed.utils_perf - WARNING - full garbage collections took 87% CPU time recently (th

CPU times: user 11min 58s, sys: 5min 48s, total: 17min 46s
Wall time: 6h 40min 6s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,params_tree_method,state
0,0,0.866717,3.566109,0.336990,logloss,0.110663,0.080941,17,289,0.400894,gpu_hist,COMPLETE
1,1,0.885717,0.003675,0.470387,logloss,0.004926,0.087824,13,294,0.759316,gpu_hist,COMPLETE
2,2,0.887100,0.732165,0.957175,logloss,0.001125,0.045880,9,286,0.947977,gpu_hist,COMPLETE
3,3,0.880850,0.097329,0.357466,logloss,1.206300,0.004286,17,38,0.671190,gpu_hist,COMPLETE
4,4,0.866200,0.008397,0.573746,logloss,0.202615,0.006200,9,205,0.932661,gpu_hist,COMPLETE
